In [1]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [2]:
data = np.load('CIFAR10_resnet50-keras_features.npz')

In [3]:
x_trn = np.vstack((data['features_training'], data['features_testing']))
y_trn = np.hstack((data['labels_training'], data['labels_testing']))

In [4]:
from sklearn.utils import shuffle
x_trn, y_trn = shuffle(x_trn, y_trn, random_state=0)

In [5]:
x_trn = np.reshape(x_trn, (-1, 2048))
x_tst, y_tst = x_trn[:100], y_trn[:100]
x_val, y_val = x_trn[100:1100], y_trn[100:1100]
x_trn, y_trn = x_trn[1100:], y_trn[1100:]

In [7]:
K = 10
dist_rand = np.load('eps0.1/dist_rand.npy')
contrast = np.load('eps0.1/contrast.npy')
dist_knn = np.load('eps0.1/dist_knn.npy')

dist_rand = np.mean(dist_rand, axis=0)
contrast = np.mean(contrast, axis=0)[K-1]
dist_knn = np.mean(dist_knn, axis=0)[K-1]

In [8]:
import numpy as np
import pdb


def lsh_function(t,x,w,b):
    # x is 1-d array
    h = np.floor((np.dot(w,x)+b)/t).astype(int)
    return h


class LSH:
    def __init__(self,n_hash_bit,n_hash_table,x_trn,y_trn,t=0.1):
        self.n_hash_bit = n_hash_bit
        self.n_hash_table = n_hash_table
        self.t = t # width of projections
        self.x_trn = x_trn
        self.y_trn = y_trn
        self.N, self.dim = x_trn.shape
        # draw w from a normal distribution (2-stable)
        self.w = np.random.normal(0, 1, (n_hash_table, n_hash_bit, self.dim))
        # draw b from U[0,t]
        self.b = np.random.uniform(0, self.t, (n_hash_table, n_hash_bit))
        self.x_trn_hash = [dict() for i in range(n_hash_table)]
        for i in tqdm(range(self.N)):
            hash_code_all = lsh_function(self.t, x_trn[i] / dist_rand, self.w, self.b)
            for l in range(n_hash_table):
                hash_code_trn = '.'.join(map(str, hash_code_all[l, :]))
                if hash_code_trn in self.x_trn_hash[l].keys():
                    self.x_trn_hash[l][hash_code_trn].append(i)
                else:
                    self.x_trn_hash[l][hash_code_trn] = [i]
#             if i % 1000 == 0:
#                 print('build hash %s'%i)

    def get_approx_KNN(self,x_tst,K):
        N_tst = x_tst.shape[0]
        x_tst_knn = np.ones((N_tst, K)) * (-1)
        nns_len = np.zeros(N_tst)
        for i_tst in tqdm(range(N_tst)):
            nns = []
            for l in range(self.n_hash_table):
                hash_code_int = lsh_function(self.t, x_tst[i_tst] / dist_rand, self.w[l, :, :], self.b[l, :])
                hash_code_test = '.'.join(map(str, hash_code_int))
                if hash_code_test in self.x_trn_hash[l].keys():
                    nns += self.x_trn_hash[l][hash_code_test]
            nns = np.unique(nns)
            num_collide_elements = len(nns)
            if len(nns) > 0:
                dist = [np.linalg.norm(self.x_trn[i] / dist_rand - x_tst[i_tst] / dist_rand, 2) for i in nns]
                dist_min_ind = nns[np.argsort(dist)]
                if num_collide_elements < K:
                    x_tst_knn[i_tst, :num_collide_elements] = dist_min_ind[:num_collide_elements]
                else:
                    x_tst_knn[i_tst, :] = dist_min_ind[:K]
            # pdb.set_trace()
            nns_len[i_tst] = len(nns)
            if i_tst % 100 == 0:
                print('get approximate knn %s'%i_tst)
        return x_tst_knn.astype(int),nns_len


    def compute_approx_shapley(self,x_tst_knn,y_tst,K):
        N_tst,K_star = x_tst_knn.shape
        # flag_sufficient = (x_tst_knn[:,-1]>=0)
        sp_approx = np.zeros((N_tst,self.N))
        for j in tqdm(range(N_tst)):
            non_nan_index = np.where(x_tst_knn[j,:]>=0)[0]
            if len(non_nan_index)== 0:
                continue
            K_tot = non_nan_index[-1]
            if K_tot == self.N:
                sp_approx[j, x_tst_knn[j, self.N - 1]] = equal(self.y_trn[x_tst_knn[j, self.N - 1]], y_tst[j]) / self.N
            for i in np.arange(K_tot - 1, -1, -1):
                sp_approx[j, x_tst_knn[j, i]] = sp_approx[j, x_tst_knn[j, i+1]] + (
                        equal(self.y_trn[x_tst_knn[j, i]], y_tst[j]) - equal(
                    self.y_trn[x_tst_knn[j, i + 1]], y_tst[j])) / K * min([K, i + 1]) / (i + 1)



        return sp_approx

In [9]:
def equal(a, b):
    return int(a == b)
    
def fine_tune(n_hash_table=10, alpha=0.5, file=False, K=1):
    t = r_vec_normalize
    n_trn = len(x_trn)
    n_hash_bit = int(np.ceil(np.log(n_trn)*alpha/ np.log(1 / f_h(1, t))))
    if file is True:
        print(n_hash_bit, file=open('eps0.1/log.txt', 'a'))
    else:
        print(n_hash_bit)
    
    start = time.time()
    lsh = LSH(n_hash_bit=n_hash_bit, n_hash_table=n_hash_table, x_trn=x_trn, y_trn=y_trn, t=t)
    runtime_build_hash = time.time() - start
    if file is True:
        print(runtime_build_hash, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_build_hash)
    
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val, K_star)
    runtime_query = time.time() - start
    if file is True:
        print(runtime_query, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val, K)
    runtime_approx_value = time.time() - start
    if file is True:
        print('it takes %s to get appox knn value' % runtime_approx_value, file=open('eps0.1/log.txt', 'a'))
    else:
        print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(val_sp_gt - sp_approx,ord=np.inf, axis=1)
    if file is True:
        print('max error %s'% np.percentile(sp_err_inf_val,90), file=open('eps0.1/log.txt', 'a'))
    else:
        print('max error %s'% np.percentile(sp_err_inf_val,90))
    return lsh

In [10]:
def fine_tune_test(n_hash_table=10, lsh=None, file=False, K=1):
    if not lsh:
        t = r_vec_normalize
        n_trn = len(x_trn)
        n_hash_bit = int(np.ceil(np.log(n_trn)*0.3/ np.log(1 / f_h(1, t))))
        print(n_hash_bit)
    
        start = time.time()
        lsh = LSH(n_hash_bit=n_hash_bit, n_hash_table=n_hash_table, x_trn=x_trn_norm, y_trn=y_trn, t=t)
        runtime_build_hash = time.time() - start
        print(runtime_build_hash)
    
    K_star = 10
    start = time.time()
    x_tst_knn_approx, nns_vec = lsh.get_approx_KNN(x_tst, K_star)
    runtime_query = time.time() - start
    if file is True:
        print(runtime_query, file=open('eps0.1/log.txt', 'a'))
    else:
        print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_tst_knn_approx, y_tst, K)
    runtime_approx_value = time.time() - start
    if file is True:
        print('it takes %s to get appox knn value' % runtime_approx_value, file=open('eps0.1/log.txt', 'a'))
    else:
        print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    if file is True:
        print('max error %s'% np.percentile(sp_err_inf_val,90), file=open('eps0.1/log.txt', 'a'))
    else:
        print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx, nns_vec

In [11]:
def val_error(K, sp_gt):
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [12]:
def test_error(K, sp_gt):
    K_star = 10
    start = time.time()
    x_tst_knn_approx, nns_vec = lsh.get_approx_KNN(x_tst, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_tst_knn_approx, y_tst, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [15]:
sp_gt = np.load('eps0.1/tst_exact_sp_gt.npy')
val_sp_gt = np.load('eps0.1/val_exact_sp_gt.npy')

In [17]:
lsh_82_05 = np.load('eps0.1/lsh_82_05.npy')

In [18]:
lsh = lsh_82_05

In [20]:
lsh = lsh[()]

In [23]:
test_error(1, sp_gt)

get approximate knn 0

26.678581953048706



it takes 0.06381607055664062 to get appox knn value
max error 0.08624021721377484


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
val_error(1, val_sp_gt)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900

255.46341681480408



it takes 0.1111290454864502 to get appox knn value
max error 0.09189972931842029


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from tqdm import tqdm_notebook as tqdm
def get_true_KNN(x_trn,x_tst):
    N = len(x_trn)
    N_tst = len(x_tst)
    x_tst_knn_gt = np.zeros((N_tst, N))
    for i_tst in tqdm(range(N_tst)):
        dist_gt = np.zeros(N)
        for i_trn in tqdm(range(N)):
            dist_gt[i_trn] = np.linalg.norm(x_trn[i_trn] - x_tst[i_tst], 2)
#             if i_trn % 100000 == 0:
#                 print(i_trn)
        x_tst_knn_gt[i_tst, :] = np.argsort(dist_gt)
    return x_tst_knn_gt.astype(int)

In [ ]:
start = time.time()
x_tst_knn_gt = get_true_KNN(x_trn, x_tst)
end1 = time.time() - start
print(end1)

In [29]:
start = time.time()
x_val_knn_gt = get_true_KNN(x_trn, x_val)
val_end1 = time.time() - start
print(val_end1)


475.9985885620117


In [30]:
def compute_single_unweighted_knn_class_shapley(x_trn,y_trn,x_tst_knn_gt,y_tst,K):
    N = x_trn.shape[0]
    N_tst = x_tst_knn_gt.shape[0]
    sp_gt = np.zeros((N_tst,N))
    for j in tqdm(range(N_tst)):
        # try:
        sp_gt[j,x_tst_knn_gt[j,-1]] = (y_trn[x_tst_knn_gt[j,-1]] == y_tst[j])/N
        # except:
        #     pdb.set_trace()
        for i in np.arange(N-2,-1,-1):
            sp_gt[j,x_tst_knn_gt[j,i]] = sp_gt[j,x_tst_knn_gt[j,i+1]] +\
                           (int(y_trn[x_tst_knn_gt[j,i]] == y_tst[j]) -
                            int(y_trn[x_tst_knn_gt[j,i+1]] == y_tst[j]))/K* min([K,i+1])/(i+1)
            # if j == 404 and i == 0:
            #     pdb.set_trace()
    return sp_gt

In [31]:
K = 2
start2 = time.time()
sp_gt = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, K)
end2 = time.time() - start2

NameError: name 'start1' is not defined

In [32]:
K = 5
start5 = time.time()
sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, 5)
end5 = time.time() - start5

In [34]:
sp_gt2 = sp_gt
sp_gt2_approx = test_error(2, sp_gt2)

get approximate knn 0

24.77258563041687



it takes 0.053353071212768555 to get appox knn value
max error 0.08624021721377484


In [35]:
sp_gt5_approx = test_error(5, sp_gt5)

get approximate knn 0

26.250773668289185



it takes 0.03290390968322754 to get appox knn value
max error 0.08108211455321042


In [36]:
sp_gt5_approx.shape

(100, 58900)

In [38]:
K = 2
start2 = time.time()
val_sp_gt2 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val, K)
end2 = time.time() - start2

In [39]:
K = 5
start5 = time.time()
val_sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_val_knn_gt, y_val, K)
end5 = time.time() - start5

In [40]:
def val_error(K, val_sp_gt):
    K_star = 10
    start = time.time()
    x_val_knn_approx, nns_vec = lsh.get_approx_KNN(x_val, K_star)
    runtime_query = time.time() - start
    print(runtime_query)
    
    start = time.time()
    sp_approx = lsh.compute_approx_shapley(x_val_knn_approx, y_val, K)
    runtime_approx_value = time.time() - start
    print('it takes %s to get appox knn value' % runtime_approx_value)
    
    sp_err_inf_val= np.linalg.norm(val_sp_gt - sp_approx,ord=np.inf, axis=1)
    print('max error %s'% np.percentile(sp_err_inf_val,90))
    return sp_approx

In [41]:
val_sp_gt5_approx = val_error(5, val_sp_gt5)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900

227.68678307533264



it takes 0.1186971664428711 to get appox knn value
max error 0.08770537330315034


In [42]:
val_sp_gt2_approx = val_error(2, val_sp_gt2)

get approximate knn 0
get approximate knn 100
get approximate knn 200
get approximate knn 300
get approximate knn 400
get approximate knn 500
get approximate knn 600
get approximate knn 700
get approximate knn 800
get approximate knn 900

245.42032408714294



it takes 0.19292879104614258 to get appox knn value
max error 0.09168467247143411
